In [ ]:
import random

import matplotlib.pyplot as plt
import numpy as np

from saxs.data_generation.processing import Processing
from saxs.data_generation.data_visualization import load_data, plot_saxs, plot_saxs_featuremap
from saxs.data_generation.generation import Generator
from saxs.data_generation import DEFAULT_CONFIG_PATH

import json

from saxs.gaussian_processing.processing_outils import read_data

with open(DEFAULT_CONFIG_PATH) as config:
    config_data = json.load(config)

In [ ]:
q, d1 = load_data(phase=config_data['phase'],
                                cubic_mesophase=config_data['cubic_mesophase'],
                                )
q = q[:498]
q_0, I_0, dI = read_data('/Users/isaigordeev/Desktop/2023/saxs/res/075775_treated_xye.csv')
I_0 = I_0[:498]
q_0 = q_0[:498]
mean = np.mean(I_0)
var = np.std(I_0)
I_0 -= mean
I_0 /= (var ** 0.5)
I_0 /= np.max(I_0)

In [ ]:
for n in random.sample(range(len(d1)), 10):
    # plot_saxs(q , d1[n])
    plt.plot(q, d1[n]/np.max(d1[n]))
    plt.plot(q_0, I_0/np.max(I_0), 'red')

plt.show()

In [ ]:
image = np.uint8(d1[0]*255)
plt.imshow(np.repeat(np.expand_dims(np.outer(image, image), -1), 3, axis=-1))

In [ ]:
image = np.uint8(I_0/np.max(I_0)*255)
plt.imshow(np.repeat(np.expand_dims(np.outer(image, image), -1), 3, axis=-1))

In [ ]:
for n in random.sample(range(len(d1)), 10):
    # plot_saxs(q , d1[n])
    plt.plot(q, d1[n]/np.max(d1[n]))
    plt.plot(q_0, I_0/np.max(I_0), 'red')

plt.show()

In [ ]:
image = np.uint8(d1[0]*255)
plt.imshow(np.repeat(np.expand_dims(np.outer(image, image), -1), 3, axis=-1))

In [ ]:
len(d1[0][68:])

In [3]:


from saxs.saxs_model.model import SAXSViT10

from torchinfo import summary #optional

from saxs.saxs_model.model_settings import IMAGE_DIM, COLOR_CHANNELS, PATCH_SIZE, ATTENTION_BLOCKS, EMBEDDING_DIM

mod = SAXSViT10(448,
                COLOR_CHANNELS,
                14,
                3,
                228,
                3072, 12, 0.1, 0, 0.1, 3)

print(mod.patch_embedding)


summary(mod,
        input_size=(1, 3, IMAGE_DIM, IMAGE_DIM),  # (batch_size, color_channels, height, width)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
        )


228
PatchEmbedding(
  (patcher): Conv2d(3, 228, kernel_size=(14, 14), stride=(14, 14))
  (flatten): Flatten(start_dim=2, end_dim=3)
)


Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
SAXSViT10 (SAXSViT10)                    [1, 3, 448, 448]     [1, 3]               233,928              True
├─PatchEmbedding (patch_embedding)       [1, 3, 448, 448]     [1, 1024, 228]       --                   True
│    └─Conv2d (patcher)                  [1, 3, 448, 448]     [1, 228, 32, 32]     134,292              True
│    └─Flatten (flatten)                 [1, 228, 32, 32]     [1, 228, 1024]       --                   --
├─Dropout (embedding_dropout)            [1, 1025, 228]       [1, 1025, 228]       --                   --
├─Sequential (transformer_encoder)       [1, 1025, 228]       [1, 1025, 228]       --                   True
│    └─TransformerEncoderLayer (0)       [1, 1025, 228]       [1, 1025, 228]       1,613,892            True
│    └─TransformerEncoderLayer (1)       [1, 1025, 228]       [1, 1025, 228]       1,613,892            True
│    └─Transformer

In [ ]:

import os
from saxs import PACKAGE_PATH, DEFAULT_PHASES_PATH
from saxs.saxs_model.model import SAXSViT10


import matplotlib.pyplot as plt
import torch
from torchvision.transforms import transforms

import saxs.saxs_model.saxs_dataset as data_setup
from saxs import PACKAGE_PATH
import json
import saxs.saxs_model.phase_prediction as phase_prediction
from saxs.saxs_model import engine
from saxs.saxs_model.model import SAXSViT
from saxs.saxs_model.model_settings import DEVICE

from torchinfo import summary #optional

from saxs.saxs_model.model_settings import IMAGE_DIM, COLOR_CHANNELS, PATCH_SIZE, ATTENTION_BLOCKS, EMBEDDING_DIM

with open(DEFAULT_PHASES_PATH, 'r') as file:  # NOTE make it better with string formatting
    phases = json.load(file)

classes = list(phases.keys())
class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}


mod = SAXSViT10(IMAGE_DIM,
                COLOR_CHANNELS,
                166,
                3,
                24,
                3072, 12, 0.1, 0, 0.1, 3)

train_saxs_batches, test_saxs_batches, saxs_phases = \
    data_setup.create_data_batches_from_dataset_files(path=os.path.join(PACKAGE_PATH, 'cache'),
                                                    transforms=None,
                                                    batch_size=32,
                                                    num_workers=0
                                                     )

# optimizer = torch.optim.Adam(params=mod.parameters(),
#                              lr=1e-3)
# loss_fn = torch.nn.CrossEntropyLoss()
#
# pretrained_vit_results = engine.train(model=mod,
#                                       train_dataloader=train_saxs_batches,
#                                       test_dataloader=test_saxs_batches,
#                                       optimizer=optimizer,
#                                       loss_fn=loss_fn,
#                                       epochs=1,
#                                       device=DEVICE)


phase_prediction.prediction_from_npy(mod,
                            'res/075773_treated_xye.csv',
                            classes,
                            )


In [ ]:
import saxs.saxs_model.phase_prediction as prediction_from_csv
from saxs.saxs_model.model import SAXSViT10
import torch

model = SAXSViT10(498,
                3,
                166,
                3,
                96,
                3072, 12, 0.1, 0, 0.1, 3)

state_dict = torch.load('model0.pth')


model.load_state_dict(state_dict)

prediction_from_csv(model, 'res/075773_treated_xye.csv')

In [5]:
import torchvision

pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT

vit_pretrained = torchvision.models.vit_b_16(pretrained_vit_weights)

<generator object Module.parameters at 0x146412040>

In [7]:

summary(vit_pretrained,
        input_size=(1, 3, 224, 224),  # (batch_size, color_channels, height, width)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
        )

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                        [1, 3, 224, 224]     [1, 1000]            768                  True
├─Conv2d (conv_proj)                                         [1, 3, 224, 224]     [1, 768, 14, 14]     590,592              True
├─Encoder (encoder)                                          [1, 197, 768]        [1, 197, 768]        151,296              True
│    └─Dropout (dropout)                                     [1, 197, 768]        [1, 197, 768]        --                   --
│    └─Sequential (layers)                                   [1, 197, 768]        [1, 197, 768]        --                   True
│    │    └─EncoderBlock (encoder_layer_0)                   [1, 197, 768]        [1, 197, 768]        7,087,872            True
│    │    └─EncoderBlock (encoder_layer_1)                   [1, 197, 768]        [1, 197, 768